# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Generate-pre-processed-FASTA-files" data-toc-modified-id="Generate-pre-processed-FASTA-files-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Generate pre-processed FASTA files</a></div><div class="lev2 toc-item"><a href="#Setup" data-toc-modified-id="Setup-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Setup</a></div><div class="lev2 toc-item"><a href="#Prepare-low-spike" data-toc-modified-id="Prepare-low-spike-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Prepare low-spike</a></div><div class="lev2 toc-item"><a href="#Prepare-high-spikes" data-toc-modified-id="Prepare-high-spikes-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Prepare high-spikes</a></div><div class="lev1 toc-item"><a href="#Simulate-MS-data" data-toc-modified-id="Simulate-MS-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Simulate MS-data</a></div><div class="lev1 toc-item"><a href="#Post-process-data" data-toc-modified-id="Post-process-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Post-process data</a></div>

# Generate pre-processed FASTA files

Using the custom-made `seqtk generate_spikein.py` script.

Dependencies:

* OpenMS binaries in PATH (version 2.2.0)

## Setup

In [1]:
threads=6
run=batch11_maybe_final
mkdir ${run}

mkdir: cannot create directory ‘batch11_maybe_final’: File exists


: 1

In [2]:
sample_names_low="l1 l2 l3"
sample_names_high="h1 h2 h3"
samples="${sample_names_low} ${sample_names_high}"

In [3]:
echo -e "name\tbiorepgroup\ttechrepgroup\tcondition" > ${run}/design.tsv
echo -e "l1\t1\t1\tlow" >> ${run}/design.tsv
echo -e "l2\t2\t1\tlow" >> ${run}/design.tsv
echo -e "l3\t3\t1\tlow" >> ${run}/design.tsv
echo -e "h1\t1\t1\thigh" >> ${run}/design.tsv
echo -e "h2\t2\t1\thigh" >> ${run}/design.tsv
echo -e "h3\t3\t1\thigh" >> ${run}/design.tsv

cat ${run}/design.tsv

name	biorepgroup	techrepgroup	condition
l1	1	1	low
l2	2	1	low
l3	3	1	low
h1	1	1	high
h2	2	1	high
h3	3	1	high


## Prepare low-spike

In [4]:
lfqtk generate_spikein_set \
    --background_fa data/uniprot_ecoli.pure.fasta \
    --spikein_fa data/uniprot_potato.pure.fasta \
    --offset_mean 0 \
    --offset_std 0 \
    --base_int 1000000 \
    --noise_std 50000 \
    --back_count 20 \
    --spike_count 5 \
    --spike_folds "1,1,1,8,8,8" \
    --out_base ${run} \
    --sample_names "l1.fa,l2.fa,l3.fa,h1.fa,h2.fa,h3.fa" \
    --verbose

4436 entries loaded from data/uniprot_ecoli.pure.fasta as background
403 entries loaded from data/uniprot_potato.pure.fasta as spike-in
20 entries picked as background, 5 as spike-in
25 entries written to batch11_maybe_final/l1.fa
20 entries picked as background, 5 as spike-in
25 entries written to batch11_maybe_final/l2.fa
20 entries picked as background, 5 as spike-in
25 entries written to batch11_maybe_final/l3.fa
20 entries picked as background, 5 as spike-in
25 entries written to batch11_maybe_final/h1.fa
20 entries picked as background, 5 as spike-in
25 entries written to batch11_maybe_final/h2.fa
20 entries picked as background, 5 as spike-in
25 entries written to batch11_maybe_final/h3.fa


# Simulate MS-data

Generate the OpenMS tool `MSSimulator`.

In [ ]:
MSSimulator -in ${run}/l1.fa -out_fm ${run}/l1.ground.featureXML

In [ ]:
time parallel -j ${threads} \
"echo \"Processing {}\"
MSSimulator \
    -in ${run}/{}.fa \
    -out_fm ${run}/{}.ground.featureXML \
    ::: ${samples}
#     -out_id ${run}/{}.ground.idXML \
#     -out ${run}/{}.mzML \
#     -out_pm ${run}/{}.centroided.mzML" \


In [ ]:
for xml in ${run}/*.featureXML; do 
    ProteinQuantifier \
        -in ${xml} \
        -peptide_out ${xml%.*}.csv
done

In [ ]:
# time parallel -j ${threads} \
# "echo \"Processing {}\"
# FeatureFinderCentroided \
#     -in ${run}/{}.centroided.mzML \
#     -out ${run}/{}.featureXML" \
#     ::: ${samples}


In [ ]:
# for sample in ${samples}; do \
#     echo "Processing sample: ${sample}"
#     IDMapper \
#         -id ${run}/${sample}.ground.idXML \
#         -in ${run}/${sample}.featureXML \
#         -out ${run}/${sample}.mapped.featureXML \
#         > ${run}/${sample}.mapped.featureXML.log
# done

Here, we do RT alignment (?) of features (with their identities mapped).

In [ ]:
# out_strings=""
# for sample in ${samples}; do
#     out_strings="${out_strings} ${run}/${sample}.mapped.aligned.featureXML"
# done

# echo ${out_strings}

# MapAlignerPoseClustering \
#     -in ${run}/*.mapped.featureXML \
#     -out ${out_strings}


In [ ]:
# # FeatureLinkerUnlabeledQT \
# #     -in ${run}/*.mapped.aligned.featureXML \
# #     -out ${run}/combined.consensusXML
    
# FeatureLinkerUnlabeledQT \
#     -in ${run}/*.mapped.featureXML \
#     -out ${run}/combined.consensusXML

# Post-process data

Extract and prepare the consensus data for normalization - transform it to an appropriate format.

In [ ]:
# TextExporter \
#     -in ${run}/combined.consensusXML \
#     -out ${run}/combined.linked_features.csv \
#     -consensus:features ${run}/combined.features.csv \
#     -consensus:elements ${run}/combined.elements.csv \
#     -consensus:centroids ${run}/combined.centroids.csv

Not sure why, but the `combined.features.csv` file included extra empty fields with numbers beyond the actual sample range. These are omitted here, but should probably be remedied upstreams instead.

In [ ]:
# s_count=$(echo ${samples} | tr " " "\n" | wc -l)
# cut_end=$(echo "${s_count} * 5 + 9" | bc)
# cut -f 1-${cut_end} ${run}/combined.features.csv \
#     > ${run}/combined.features.sub.csv

In [ ]:
# util_scripts/openms_to_normalyzer.py \
#     -i ${run}/combined.features.sub.csv \
#     -o ${run}/combined.final.tsv \
#     --design ${run}/design.tsv